In [1]:
import random
from Data_import import data_frame
from model_evaluation import *
from _xgbod import hazard_wind_detection_XGBOD
np.random.seed(1)
random.seed(1)
Data = data_frame()
calm_train = Data.calm_train
pilot_train = Data.pilot_semi_learning_generator(semi_ratio = 1)
X_unlabel = Data.unlabeled_data_v_r[::500,::]
Y_unlabel = np.zeros((X_unlabel.shape[0],1))
train_x = np.vstack((calm_train, pilot_train))#labeled data
train_y = np.hstack((np.zeros(calm_train.shape[0]), np.zeros(pilot_train.shape[0]) + 1))
model = hazard_wind_detection_XGBOD()
model.train_base_unsupervise_classifiers(X_unlabel)

In [2]:
import numpy as np
from models.utility import print_baseline
from xgboost.sklearn import XGBClassifier
for count_imbalance, beta_imbalance in enumerate(np.array([800])):#1,10,100,200,400,600,800,
    # calm_test_x = Data.calm_test_generator(calm_ratio=beta_imbalance)
    index = Data.calm_ratios.index(beta_imbalance)
    calm_test_x = Data.calm_test_generator(calm_ratio_index=index)
    test_x = np.vstack((calm_test_x, Data.pilot_test_v_r))
    test_y = np.hstack((np.zeros(calm_test_x.shape[0]), np.zeros(Data.pilot_test_v_r.shape[0]) + 1))
    X_train_new, test_x_new, roc_list = model.unsupervised_classifiers_decision_functions(train_x, test_x, test_y)
    X_train_all = np.concatenate((train_x, X_train_new), axis=1)
    test_x_all = np.concatenate((test_x, test_x_new), axis=1)
    clf = XGBClassifier()
    # semi-supervised
    clf.fit(X_train_all, train_y.ravel())
    y_pred = clf.predict_proba(test_x_all)
    semisupervised_AUC, _ = calculate_metrics(decision_values=y_pred[:, 1], truth=test_y, hazard_id=1)

    print(f"=====================beta_imbalance = {beta_imbalance}=====================")
    print(f'semi-superviseAUC: {semisupervised_AUC}')


=====================beta_imbalance = 800=====================
semi-superviseAUC: 0.906476430933512
